# Reproduce Table 1
- Py kernel

In [1]:
import pandas as pd
import numpy as np
import pingouin as pg

from scipy import stats

/Users/brinkley97/opt/anaconda3/envs/new_tiles_day_night/lib/python3.9/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(
/Users/brinkley97/opt/anaconda3/envs/new_tiles_day_night/lib/python3.9/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


# Load Data

In [2]:
base = "/Users/brinkley97/Documents/development/lab-kcad/"
path_to_file = "datasets/tiles_dataset/table_1/"
name_of_file = "nurse_data.csv"
file = base + path_to_file + name_of_file

In [3]:
def load_data(file):
    
    original_data = pd.read_csv(file)
    copy_of_data = original_data.copy()
    
    return copy_of_data

In [4]:
nurse_df = load_data(file)
# nurse_df

In [5]:
day_shift = nurse_df['Shift'] == 'Day shift'
day_df = nurse_df.loc[day_shift]
# day_df.head(5)

In [6]:
night_shift = nurse_df['Shift'] == 'Night shift'
night_df = nurse_df.loc[night_shift]
# night_df.head(5)

# Load Generated Specific Questions

In [7]:
generated_sq_file = "TGN10Plus/generateSpecificQuestions.ipynb"
table_1_specific_questions = base + generated_sq_file
# table_1_specific_questions

In [8]:
# %load table_1_specific_questions
/Users/brinkley97/Documents/development/lab-kcad/TGN10Plus/generateSpecificQuestions.ipynb

In [9]:
%run "../generateSpecificQuestions.ipynb"

In [10]:
table_1_demographic_sqs

['what are differences in primarily *day-shift* nurses and primarily *night-shift* nurses for *Gender* ?',
 'what are differences in primarily *day-shift* nurses and primarily *night-shift* nurses for *Age* ?',
 'what are differences in primarily *day-shift* nurses and primarily *night-shift* nurses for *Educ* ?',
 'what are differences in primarily *day-shift* nurses and primarily *night-shift* nurses for *native_lang* ?',
 'what are differences in primarily *day-shift* nurses and primarily *night-shift* nurses for *Female* ?',
 'what are differences in primarily *day-shift* nurses and primarily *night-shift* nurses for *Male* ?',
 'what are differences in primarily *day-shift* nurses and primarily *night-shift* nurses for *< 40 Years* ?',
 'what are differences in primarily *day-shift* nurses and primarily *night-shift* nurses for *>= 40 Years* ?',
 'what are differences in primarily *day-shift* nurses and primarily *night-shift* nurses for *Some college or College* ?',
 'what are di

# Run analysis

## Fisher Odds Test - Table 1

In [11]:
def fisher_odds_p_value(day_shift_df, night_shift_df, specific_demographic_ontology_key, demo_group_1, demo_group_2):
    '''Calculate the fisher oddsration and p-values using stats.fisher_exact function
    
    Arguments:
    day_shift_df -- Pandas DataFrame
    night_shift_df -- Pandas DataFrame
    demo -- str which is used as a column in the two Pandas DataFrames 
    demo_group_1 -- str which is used as a column in the two Pandas DataFrames 
    demo_group_2 -- str which is used as a column in the two Pandas DataFrames 
    
    Return: 
    oddsratio and pvalue
    '''
    
    tabel_df = pd.DataFrame(index=['day', 'night'], columns=[demo_group_1, demo_group_2])
    tabel_df.loc['day', demo_group_1] = len(day_shift_df.loc[day_shift_df[specific_demographic_ontology_key] == demo_group_1])
    tabel_df.loc['day', demo_group_2] = len(day_shift_df.loc[day_shift_df[specific_demographic_ontology_key] == demo_group_2])
    tabel_df.loc['night', demo_group_1] = len(night_shift_df.loc[night_shift_df[specific_demographic_ontology_key] == demo_group_1])
    tabel_df.loc['night', demo_group_2] = len(night_shift_df.loc[night_shift_df[specific_demographic_ontology_key] == demo_group_2])
    oddsratio, pvalue = stats.fisher_exact(np.array(tabel_df))
    
    return 'oddsratio : %.3f, pvalue: %.3f \n' % (oddsratio, pvalue)

In [12]:
def table_one_demo_ontologies(demo_specific_questions):
    store_p_values = []

    for demo_specific_question in demo_specific_questions:
        
        demographic_ontology_keys = list(ontology_mappings["demographic"])
        demographic_ontology_values = list(ontology_mappings.values())[2]
        
        for specific_demographic_ontology_key in demographic_ontology_keys:
            
            if specific_demographic_ontology_key in demo_specific_question.split("*"):
                demo_group_1 = demographic_ontology_values[specific_demographic_ontology_key][0]
                demo_group_2 = demographic_ontology_values[specific_demographic_ontology_key][1]
               
                p_value = fisher_odds_p_value(day_df, night_df, specific_demographic_ontology_key=specific_demographic_ontology_key, demo_group_1=demo_group_1, demo_group_2=demo_group_2)
                store_p_values.append(specific_demographic_ontology_key)
                store_p_values.append(p_value)
                
            else:
                continue
 
    return store_p_values

In [13]:
table_one_demo_ontologies(table_1_demographic_sqs)

['Gender',
 'oddsratio : 1.188, pvalue: 0.829 \n',
 'Age',
 'oddsratio : 0.517, pvalue: 0.160 \n',
 'Educ',
 'oddsratio : 1.967, pvalue: 0.215 \n',
 'native_lang',
 'oddsratio : nan, pvalue: 1.000 \n']

In [ ]:
behav_specific_questions = list(t_test_questions.values())[1]
behav_specific_questions

In [ ]:
def ttest_p_value(all_df, first_df, second_df, behav, print_col, end=False, stat='num'):
    
    p_values = []
    result = pg.ttest(np.array(first_df[behav].dropna()), np.array(second_df[behav].dropna()))
    p_values.append(behav)
    
    p = result['p-val'].values[0]
    p_values.append(p)
    
    return p_values

In [ ]:
def table_one_behav(behav_specific_questions):
    p_values = []
    behav_values = list(key_words_in_study.values())[3]
    # print("behav_values : ", behav_values)
    
    behavs = list(key_words_in_study["behavioral"])
    # print(behavs)
    
    for behav_specific_question in behav_specific_questions:
        # print("\n", behav_specific_question)
        
        for behav in behavs:
            # print("\nbehav : ", behav)
            
            if behav in behav_specific_question.split("*"):
                p_value = ttest_p_value(nurse_df, day_df, night_df, behav, behav, end=False, stat='num')
                # print(p_value)
                p_values.append(p_value)
                behavs.remove(behav)
                # print("\n", behavs)
                
    return p_values

In [ ]:
table_one_behav(behav_specific_questions)